<a href="https://colab.research.google.com/github/kumarprakhar14/Spark-Detection-System/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


def main():
    cap = cv2.VideoCapture('/content/Spark_Detection.mp4')  # Or use 0 for webcam

    # Initialize variables for motion detection
    ret, frame1 = cap.read()
    if not ret:
        print("Error reading video file")
        return

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (15, 15), 0)

    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.GaussianBlur(gray2, (15, 15), 0)

        # Compute absolute difference between current frame and previous frame
        frame_diff = cv2.absdiff(gray1, gray2)

        # Apply threshold to detect significant changes
        _, motion_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)

        # Optional: Apply morphological operations to enhance detection
        kernel = np.ones((5, 5), np.uint8)
        motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel)
        motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel)

        # Find contours of motion regions
        contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw rectangles around detected motion regions
        for contour in contours:
            if cv2.contourArea(contour) > 100:  # Adjust area threshold as needed
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Display the frame with detected motion
        cv2_imshow(frame2)

        # Update previous frame
        gray1 = gray2

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
